In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [59]:
#define function that takes a list of html and returns list of text elements
def to_text_list(list):
    new_list = []
    for item in list:
        new_list.append(item.text)
    return new_list

#function that take table elements from html and returns it in a dataframe
def html_table_to_df(sheet_elements):
    sheet_headers = to_text_list(sheet_elements[0].find_all("td"))
    
    df_lift = pd.DataFrame(columns = sheet_headers)
    
    for i in range(1,len(sheet_elements)):
        sheet_data = to_text_list(sheet_elements[i].find_all("td"))
        df_lift.loc[len(df_lift)] = sheet_data
    
    return df_lift

#define function that takes input of link to ski resort, state, and country and outputs a dataframe
def df_from_resort_link(URL, resort, state, country):
    page = requests.get(URL)

    #create a beautifulsoup object with the page content and choose the appropriate parser
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find("div", class_="entry-content")

    #get the link to the google sheet
    google_sheet = results.find("a")['href']

    google_page = requests.get(google_sheet)
    google_soup = BeautifulSoup(google_page.content, "html.parser")

    google_table = google_soup.find("tbody")
    sheet_elements = google_table.find_all("tr")

    df_lifts = html_table_to_df(sheet_elements)

    #insert column for country and state to the dataframe
    df_lifts.insert(0, "Country", [country]*(len(sheet_elements)-1), True)
    df_lifts.insert(1, "State", [state]*(len(sheet_elements)-1), True)
    df_lifts.insert(2, "Resort", [resort]*(len(sheet_elements)-1), True)
    return df_lifts

    #define function that takes input of state name, country and state url and returns a data frame for all of the resort data in that state.
def df_from_state(URL, state_name, country):
    page = requests.get(URL)

    #create a beautifulsoup object with the page content and choose the appropriate parser
    soup = BeautifulSoup(page.content, "html.parser")

    # grabs container for list of resorts
    results = soup.find("div", class_="entry-content")

    ski_resorts = results.find_all("ul")[0].find_all("a")

    # make dataframe to collect skilift data from all resorts
    df_resorts = pd.DataFrame()

    for resort in ski_resorts:
        resort_name = resort.text.strip()
        print(resort_name)
        resort_link = resort['href']
        df_resorts = pd.concat([df_resorts, df_from_resort_link(resort_link, 
                                                                resort_name, 
                                                                state_name, 
                                                                country)])

    return df_resorts  

In [3]:
URL = "https://liftblog.com/united-states/"
page = requests.get(URL)

#create a beautifulsoup object with the page content and choose the appropriate parser
soup = BeautifulSoup(page.content, "html.parser")

# grabs container for job postings
results = soup.find(id="post-1491")

#there are two lists on this page.  first is the states and second is sharing links
states = results.find_all("ul")[0].find_all("a")


for state in states:
    state_name = state.text.strip()
    state_link = state['href']
    print(state_name)
    print(state_link)



Alabama
https://liftblog.com/alabama/
Alaska
https://liftblog.com/alaska/
Arizona
https://liftblog.com/arizona/
California
https://liftblog.com/california/
Colorado
https://liftblog.com/colorado/
Connecticut
https://liftblog.com/connecticut/
Florida
https://liftblog.com/florida/
Georgia
https://liftblog.com/georgia/
Idaho
https://liftblog.com/idaho/
Illinois
https://liftblog.com/illinois/
Indiana
https://liftblog.com/indiana/
Iowa
https://liftblog.com/iowa/
Kansas
https://liftblog.com/kansas/
Kentucky
https://liftblog.com/kentucky/
Maine
https://liftblog.com/maine/
Maryland
https://liftblog.com/wisp-md/
Massachusetts
https://liftblog.com/massachusetts/
Michigan
https://liftblog.com/michigan/
Minnesota
https://liftblog.com/minnesota/
Mississippi
https://liftblog.com/mississippi/
Missouri
https://liftblog.com/missouri/
Montana
https://liftblog.com/montana/
Nebraska
https://liftblog.com/nebraska/
Nevada
https://liftblog.com/nevada/
New Hampshire
https://liftblog.com/new-hampshire
New Jers

In [60]:
df_from_state("https://liftblog.com/montana/", 'Montana', 'United States')

Bear Canyon
Bear Paw
Big Sky
Blacktail Mountain
Bridger Bowl
Great Divide
Lookout Pass
Lost Trail
Marshall Mountain
Maverick Mountain
Montana Snowbowl
Red Lodge Mountain
Showdown
Ski Discovery
Teton Pass
Turner Mountain
Whitefish Mountain
Yellowstone Club


,Country,State,Resort,Status,Lift Name,Type,Manufacturer,Years of Operation,Capacity,Vertical Rise,Length,Horsepower,Line Speed,Chairs,Towers,Drive,Tension,Ride Time,Notes
0,United States,Montana,Bear Canyon,Operating,T-Bar,T-Bar,Hall,1988-,,590,,,,,13,Bottom,Top,,Relocated
0,United States,Montana,Bear Paw,Operating,Chairlift,Double,Riblet,1977-,720,877,2465,,450,72,13,Bottom,Bottom,5.5,Many SLI parts
0,United States,Montana,Big Sky,Operating,Bear Back,Platter,Poma,1993-,,300,900,,,14,4,Bottom,Bottom,,Relocated
1,United States,Montana,Big Sky,Operating,Cabin,Triple,Doppelmayr CTEC,2005-,1200,136,1691,40,500,24,7,Top,Bottom,3.4,
2,United States,Montana,Big Sky,Operating,Cascade,Triple,CTEC,2002-,,448,3295,,,110,16,Top,Bottom,,"Relocated from Crystal Mountain, WA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,United States,Montana,Yellowstone Club,Operating,Pine Ridge,Pulse Quad,Doppelmayr CTEC,2007-,490,574,4326,200,1000,20,17,Top,Bottom,4.3,Bubbles
17,United States,Montana,Yellowstone Club,Operating,River Crossing,Double,Doppelmayr,2001-,300,18,676,30,300,9,2,Top,None,2.3,
18,United States,Montana,Yellowstone Club,Operating,Silvertip,High Speed Quad,Doppelmayr,2019-,1800,1235,4762,500,1000,43,17,Top,Bottom,4.8,Bubbles
19,United States,Montana,Yellowstone Club,Operating,Southern Cross,Triple,Doppelmayr CTEC,2007-,600,290,1276,60,450,,6,Top,Bottom,2.8,


In [56]:
df_resorts

,Country,State,Resort,Status,Lift Name,Type,Manufacturer,Years of Operation,Capacity,Vertical Rise,Length,Horsepower,Line Speed,Chairs,Towers,Drive,Tension,Ride Time,Notes
0,United States,Montana,Bear Canyon,Operating,T-Bar,T-Bar,Hall,1988-,,590,,,,,13,Bottom,Top,,Relocated
0,United States,Montana,Bear Paw,Operating,Chairlift,Double,Riblet,1977-,720,877,2465,,450,72,13,Bottom,Bottom,5.5,Many SLI parts
0,United States,Montana,Big Sky,Operating,Bear Back,Platter,Poma,1993-,,300,900,,,14,4,Bottom,Bottom,,Relocated
1,United States,Montana,Big Sky,Operating,Cabin,Triple,Doppelmayr CTEC,2005-,1200,136,1691,40,500,24,7,Top,Bottom,3.4,
2,United States,Montana,Big Sky,Operating,Cascade,Triple,CTEC,2002-,,448,3295,,,110,16,Top,Bottom,,"Relocated from Crystal Mountain, WA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,United States,Montana,Showdown,Operating,Prospector,Double,Stadeli,1970-,760,1332,6267,,500,156,27,Bottom,Bottom,12.5,Midstation
2,United States,Montana,Showdown,Operating,Sluice Goose Caboose,Double,Yan,2006-,960,634,3151,,400,126,12,Bottom,Bottom,7.9,"Relocated from Alta, UT"
3,United States,Montana,Showdown,Removed,Poma,Platter,Poma,1963-2006,,400,1200,,,,,,,,
4,United States,Montana,Showdown,Removed,T-Bar,T-Bar,Constam,1963-1978,1100,975,4060,,,,,,,,


AttributeError: 'NoneType' object has no attribute 'head'

In [42]:
['state']*len(sheet_elements)

['state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state']